In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [2]:
from google.colab import files
import pandas as pd
import io

uploaded = files.upload()  # سيفتح نافذة لاختيار الملفات من جهازك



Saving TOI_2025.09.30_12.03.40.csv to TOI_2025.09.30_12.03.40.csv


In [3]:
from google.colab import files
import pandas as pd
import io

uploaded = files.upload()  # سيفتح نافذة لاختيار الملفات من جهازك



Saving cumulative_2025.09.30_12.52.38.csv to cumulative_2025.09.30_12.52.38.csv


In [4]:
# ==============================
# 1) قراءة البيانات
# ==============================
tess_df = pd.read_csv("TOI_2025.09.30_12.03.40.csv", comment='#')
kepler_df = pd.read_csv("cumulative_2025.09.30_12.52.38.csv", comment='#')

In [5]:
# ==============================
# 2) تجهيز عمود flux
# ==============================
for df in [tess_df, kepler_df]:
    if "rmag" in df.columns:
        df["flux"] = 10**(-0.4 * df["rmag"])
    elif "vmag" in df.columns:
        df["flux"] = 10**(-0.4 * df["vmag"])
    else:
        df["flux"] = np.random.rand(len(df))

In [8]:
print(df.columns.tolist())


['toi', 'tid', 'tfopwg_disp', 'rastr', 'ra', 'decstr', 'dec', 'st_pmra', 'st_pmraerr1', 'st_pmraerr2', 'st_pmralim', 'st_pmdec', 'st_pmdecerr1', 'st_pmdecerr2', 'st_pmdeclim', 'pl_tranmid', 'pl_tranmiderr1', 'pl_tranmiderr2', 'pl_tranmidlim', 'pl_orbper', 'pl_orbpererr1', 'pl_orbpererr2', 'pl_orbperlim', 'pl_trandurh', 'pl_trandurherr1', 'pl_trandurherr2', 'pl_trandurhlim', 'pl_trandep', 'pl_trandeperr1', 'pl_trandeperr2', 'pl_trandeplim', 'pl_rade', 'pl_radeerr1', 'pl_radeerr2', 'pl_radelim', 'pl_insol', 'pl_insolerr1', 'pl_insolerr2', 'pl_insollim', 'pl_eqt', 'pl_eqterr1', 'pl_eqterr2', 'pl_eqtlim', 'st_tmag', 'st_tmagerr1', 'st_tmagerr2', 'st_tmaglim', 'st_dist', 'st_disterr1', 'st_disterr2', 'st_distlim', 'st_teff', 'st_tefferr1', 'st_tefferr2', 'st_tefflim', 'st_logg', 'st_loggerr1', 'st_loggerr2', 'st_logglim', 'st_rad', 'st_raderr1', 'st_raderr2', 'st_radlim', 'toi_created', 'rowupdate', 'flux']


In [48]:
# إزالة المسافات والحروف المخفية وتحويلها لـ lowercase
df.columns = df.columns.str.strip().str.lower()

# تحقق من الأعمدة
print(df.columns.tolist())

['kepid', 'kepoi_name', 'kepler_name', 'koi_disposition', 'koi_pdisposition', 'koi_score', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2', 'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact', 'koi_impact_err1', 'koi_impact_err2', 'koi_duration', 'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1', 'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2', 'koi_teq', 'koi_teq_err1', 'koi_teq_err2', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2', 'koi_model_snr', 'koi_tce_plnt_num', 'koi_tce_delivname', 'koi_steff', 'koi_steff_err1', 'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2', 'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec', 'koi_kepmag', 'flux', 'label', 'time']


In [50]:
# استخدم pl_tranmid كـ time
if "pl_tranmid" in df.columns:
    df["time"] = df["pl_tranmid"]
else:
    df["time"] = np.arange(len(df))  # إذا لم يكن موجود، استخدم تسلسل للتجريب


In [51]:
# تنظيف أسماء الأعمدة: إزالة المسافات والأحرف الغريبة وتحويلها لـ lowercase
df.columns = df.columns.str.strip().str.lower()

# عرض الأعمدة للتأكد
print(df.columns.tolist())


['kepid', 'kepoi_name', 'kepler_name', 'koi_disposition', 'koi_pdisposition', 'koi_score', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2', 'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact', 'koi_impact_err1', 'koi_impact_err2', 'koi_duration', 'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1', 'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2', 'koi_teq', 'koi_teq_err1', 'koi_teq_err2', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2', 'koi_model_snr', 'koi_tce_plnt_num', 'koi_tce_delivname', 'koi_steff', 'koi_steff_err1', 'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2', 'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec', 'koi_kepmag', 'flux', 'label', 'time']


In [53]:
# البحث عن الأعمدة المهمة لتوليد label
for c in df.columns:
    if 'tran' in c or 'rade' in c or 'dep' in c:
        print(c)


koi_depth
koi_depth_err1
koi_depth_err2


In [84]:
def generate_label(row):
    depth = row['koi_depth']
    radius = row['pl_rade'] if 'pl_rade' in df.columns else 0
    # تغيير الحدود بعد الفاصلة
    if depth < 0.001 or radius < 0.7:
        return 0  # No Planet
    elif depth < 0.01 or radius < 2.5:
        return 1  # Candidate
    else:
        return 2  # Confirmed

df['label'] = df.apply(generate_label, axis=1)











In [85]:

# ==============================
# 5) دمج البيانات
# ==============================
merged_df = pd.concat([tess_df, kepler_df], ignore_index=True)

In [86]:
# ==============================
# 6) تجهيز البيانات للموديل
# ==============================
X = merged_df[["time", "flux"]].values
y = merged_df["label"].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = np.reshape(X_scaled, (X_scaled.shape[0], 1, X_scaled.shape[1]))

In [87]:
# ==============================
# 7) بناء موديل LSTM
# ==============================
model = Sequential()
model.add(LSTM(64, input_shape=(X_scaled.shape[1], X_scaled.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32))
model.add(Dropout(0.2))
model.add(Dense(3, activation="softmax"))

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_16 (LSTM)                  │ (None, 1, 64)          │        17,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 1, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_17 (LSTM)                  │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,667 (115.89 KB)

 Trainable params: 29,667 (115.89 KB)

 Non-trainable params: 0 (0.00 B)

In [88]:

# ==============================
# 8) تدريب الموديل
# ==============================
history = model.fit(X_scaled, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/20
432/432 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.6230 - loss: 0.8180 - val_accuracy: 1.0000 - val_loss: 0.0012
Epoch 2/20
432/432 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6214 - loss: 0.6230 - val_accuracy: 1.0000 - val_loss: 3.8619e-04
Epoch 3/20
432/432 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.6336 - loss: 0.6079 - val_accuracy: 1.0000 - val_loss: 2.3478e-04
Epoch 4/20
432/432 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6280 - loss: 0.6142 - val_accuracy: 1.0000 - val_loss: 1.4387e-04
Epoch 5/20
432/432 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6267 - loss: 0.6106 - val_accuracy: 1.0000 - val_loss: 1.3011e-04
Epoch 6/20
432/432 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6350 - loss: 0.6032 - val_accuracy: 1.0000 - val_loss: 8.5252e-05
Epoch 7/20
432/432 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.6224 - loss: 0.6175 - val_accuracy: 1.0000 - val_loss: 6.1998e-05
Epoch 8/20
432/432 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.6268 - loss: 0

In [89]:

# ==============================
# 9) تقييم دقة الموديل الكامل
# ==============================
y_pred = np.argmax(model.predict(X_scaled), axis=-1)
accuracy = accuracy_score(y, y_pred)
print(f"✅ Full Model Accuracy (TESS + Kepler): {accuracy*100:.2f}%")

540/540 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
✅ Full Model Accuracy (TESS + Kepler): 70.21%


In [90]:

# ==============================
# 10) التنبؤ باستخدام بيانات جديدة
# ==============================
new_data = np.array([[10**(-0.4*12.5), 1.2]])  # مثال: flux و pl_rade
new_data_scaled = scaler.transform(new_data)
new_data_scaled = np.reshape(new_data_scaled, (new_data_scaled.shape[0], 1, new_data_scaled.shape[1]))

prediction = np.argmax(model.predict(new_data_scaled), axis=-1)[0]

labels_map = {0: "No Planet", 1: "Candidate", 2: "Confirmed"}
print("🔮 Prediction:", labels_map[prediction])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
🔮 Prediction: No Planet
